In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor  # Import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

# Read the dataset
df = pd.read_csv("E:\\Downloads\\Dataset (1).csv")

# Exclude unnecessary columns
df = df.drop(columns=['Unnamed: 0'])

# Encode categorical variables
df = pd.get_dummies(df, columns=['gender', 'activity_level', 'BMI_tags', 'Label'], drop_first=False)

# Separate features (X) and target variable (y)
X = df.drop(columns=['calories_to_maintain_weight'])
y = df['calories_to_maintain_weight']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model with Random Forest Regressor
model = RandomForestRegressor(random_state=42, n_estimators=100, max_depth=5, min_samples_split=10, min_samples_leaf=5)
model.fit(X_train, y_train)

# Save the model
save_path = "C:/Users/786/Documents/Imanpython/model.pkl"
try:
    joblib.dump(model, save_path)
    print(f"Model saved successfully at: {save_path}")
except Exception as e:
    print("Error during model save:", e)

# Load the model
try:
    loaded_model = joblib.load(save_path)
    print("Model loaded successfully!")
except Exception as e:
    print("Error during model load:", e)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
accuracy = r2 * 100

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2):", r2)
print("Accuracy (in %):", accuracy, "%")

# Show the actual vs predicted values
results_df = pd.DataFrame({
    'Actual Values': y_test,
    'Predicted Values': y_pred
})

# Print the first few results to see them
print("\nActual vs Predicted Values:")
print(results_df.head())

# Visualizing the predictions vs true values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, color='blue')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')
plt.title("True vs Predicted values")
plt.xlabel("True Values")
plt.ylabel("Predicted Values")
plt.show()

# Visualizing the residuals (error distribution)
residuals = y_test - y_pred
plt.figure(figsize=(8, 6))
sns.histplot(residuals, kde=True, color='green')
plt.title("Residuals Distribution")
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.show()

# Feature importance plot (if you're using random forest)
feature_importances = model.feature_importances_
features = X.columns

plt.figure(figsize=(10, 6))
sns.barplot(x=features, y=feature_importances)
plt.title("Feature Importance")
plt.xlabel("Features")
plt.ylabel("Importance")
plt.xticks(rotation=90)
plt.show()

# Prepare the input features for new prediction
age = int(input("Enter age: "))
weight_kg = float(input("Enter weight in kg: "))
height_cm = float(input("Enter height in cm: "))

# Convert height from cm to m
height_m = height_cm / 100

gender = input("Enter gender (M/F): ")

# Calculate BMI
BMI = weight_kg / (height_m ** 2)

# Calculate BMR
if gender.upper() == 'M':
    BMR = 10 * weight_kg + 6.25 * height_m * 100 - 5 * age + 5
else:
    BMR = 10 * weight_kg + 6.25 * height_m * 100 - 5 * age - 161

# Assign BMI tags
if BMI < 18.5:
    BMI_tags = 'Underweight'
    label = 'Underweight'
elif 18.5 <= BMI < 25:
    BMI_tags = 'Normal weight'
    label = 'Normal weight'
elif 25 <= BMI < 30:
    BMI_tags = 'Overweight'
    label = 'Overweight'
else:
    BMI_tags = 'Obese'
    label = 'Obese'

# Prompt for activity level with more detailed descriptions
print("\nActivity Levels (Choose the one that best fits your lifestyle):")
print("1: Sedentary - If you spend most of the day sitting (e.g., desk job) or lying down with little to no physical activity.")
print("2: Lightly Active - If you occasionally do light activities like walking for about 20-30 minutes, light stretching, or casual exercise 1-3 times a week.")
print("3: Moderately Active - If you engage in moderate physical activities such as jogging, cycling, or playing sports at least 3-5 days a week.")
print("4: Very Active - If you work out intensely 6-7 days a week, doing activities like running, intense weightlifting, or high-intensity exercises.")
print("5: Extra Active - If your job requires significant physical effort (like manual labor) or you're an athlete who trains intensely every day.")

activity_choice = int(input("Choose your activity level (1-5): "))

# Input validation for activity level
while activity_choice not in [1, 2, 3, 4, 5]:
    print("Invalid choice! Please enter a number between 1 and 5.")
    activity_choice = int(input("Choose your activity level (1-5): "))

# Map activity multipliers
activity_multipliers = {
    1: 1.2,    # Sedentary
    2: 1.375,  # Lightly Active
    3: 1.55,   # Moderately Active
    4: 1.725,  # Very Active
    5: 1.9     # Extra Active
}
activity_multiplier = activity_multipliers.get(activity_choice, 1.2)  # Default to Sedentary

# Adjust caloric needs
adjusted_calories = BMR * activity_multiplier

# Display the output
print(f"\nYour BMI is: {BMI:.2f}")
print(f"Your BMR is: {BMR:.2f}")
print(f"Your estimated daily caloric needs based on activity level: {adjusted_calories:.2f}")
print(f"You are categorized as: {label}")

# Automated guidance with safe calorie adjustments
if label == 'Underweight':
    print("You need to gain weight. Consume approximately {:.2f} calories per day.".format(adjusted_calories + 500))
elif label == 'Normal weight':
    print("You are at a normal weight. Maintain your weight by consuming {:.2f} calories per day.".format(adjusted_calories))
else:
    # Ensuring the calorie deficit is safe
    if adjusted_calories - 500 < 1200:  # For women, we ensure not dropping below 1200 calories/day
        print("You need to lose weight. Consume approximately {:.2f} calories per day.".format(1200))
    else:
        print("You need to lose weight. Consume approximately {:.2f} calories per day.".format(adjusted_calories - 500))
